In [16]:

import plotly.express as px
import pandas as pd


df = pd.DataFrame(
            {   
                "cell_number": [0,0,0,0,1,1,1,1],
                "top": [0.4, 0.3, 0.245, 0.1, 0.41, 0.3, 0.245, 0.1],
                "bottom": [0.3, 0.245, 0.1, -0.5, 0.3, 0.245, 0.1, -0.5],
                "type": ["settled", "settled", "settled", "bed", "settled", "settled", "settled", "bed"],
                "name": ["2", "1", "0", "existing", "2", "1", "0", "existing"],
            }
        )
        
df['thickness'] = abs(df['bottom'] - df['top'])
df['thickness2'] = df['thickness']
total_thicknesses = df.groupby('cell_number').sum(numeric_only=True)['thickness2']
total_thicknesses.rename('total_thickness', inplace=True)

df = pd.merge(df, total_thicknesses, on='cell_number')

print(df)
print(total_thicknesses)
        
fig = px.bar(
            df,
            x="cell_number",
            y="thickness",
            base="bottom",
            color='name',
            hover_name='cell_number',
            hover_data={
                'top': True,
                'bottom': True,
                'thickness': True, # For some reason thickness is displaying the wrong value (top)
                'cell_number': True,
                'name': True,
                'thickness2': True,
                'total_thickness': True
            },
        )

print(fig.data[0].hovertemplate)
fig.update_traces(hovertemplate="<b>Cell: %{hovertext}</b><br><br>Name=%{customdata[1]:.3f}<br>Top=%{y}<br>Bottom=%{base:.3f}<br>Thickness: %{customdata[2]:.3f}<br>Total thickness: %{customdata[3]:.3f}<br><extra></extra>")
fig.show()

   cell_number    top  bottom     type      name  thickness  thickness2  \
0            0  0.400   0.300  settled         2      0.100       0.100   
1            0  0.300   0.245  settled         1      0.055       0.055   
2            0  0.245   0.100  settled         0      0.145       0.145   
3            0  0.100  -0.500      bed  existing      0.600       0.600   
4            1  0.410   0.300  settled         2      0.110       0.110   
5            1  0.300   0.245  settled         1      0.055       0.055   
6            1  0.245   0.100  settled         0      0.145       0.145   
7            1  0.100  -0.500      bed  existing      0.600       0.600   

   total_thickness  
0             0.90  
1             0.90  
2             0.90  
3             0.90  
4             0.91  
5             0.91  
6             0.91  
7             0.91  
cell_number
0    0.90
1    0.91
Name: total_thickness, dtype: float64
<b>%{hovertext}</b><br><br>name=%{customdata[1]}<br>bottom=%{base

In [5]:
import plotly.graph_objects as go
from cmap import Colormap
import numpy as np

cm = Colormap('viridis')
df["name"] = pd.to_numeric(df["name"], errors="coerce")

n_names = len(df["name"].unique()) - 1
colours = cm(np.linspace(0,1,n_names))

fig = go.Figure()
for n, df1 in df.groupby("name", dropna=False):
    if pd.isna(n):
        colour = ['#2e2e2e'] * 2
        name = 'seabed'
    else:
        colour = colours[int(n)]
        colour = [f"rgb{colour[0], colour[1], colour[2]}"]*2
        name = f"Settled: {int(n)}"
    
    fig.add_trace(
        go.Bar(
            x=df1["cell_number"],
            y=df1["thickness"],
            base=df1["bottom"],
            marker_color=colour,
            name=name,
            hovertemplate="Top: %{y:.2f}m"
            + "<br>Bottom: %{base:.2f}"
            + "<br>Thickness: %{customdata[0]:,.2f}"
            + "<br><b>Cell</b>: %{x}<br>",
            customdata= df1[["thickness"]]
        )
    )

fig.update_layout(barmode="stack")
fig.show()


In [6]:
import plotly.graph_objects as go

# Although this method works you cannot specify a name for each horizon.


df["name"] = pd.to_numeric(df["name"], errors="coerce")

fig = go.Figure()
print(df)
fig.add_trace(
    go.Bar(
        x=df["cell_number"],
        y=df["thickness"],
        base=df["bottom"],
        marker_color=df["name"],
        hovertemplate="Top: %{y:.4f}m"
            + "<br>Bottom: %{base:.4f}"
            + "<br>Thickness: %{customdata[0]:,.4f}"
            + "<br><b>Cell</b>: %{x}<br>",
            customdata= df[["thickness"]],
    )
)
fig.update_layout(barmode="stack")
fig.show()

   cell_number    top  bottom     type  name  thickness
0            0  0.410   0.300  settled   2.0      0.110
1            0  0.300   0.245  settled   1.0      0.055
2            0  0.245   0.100  settled   0.0      0.145
3            0  0.100  -0.500      bed   NaN      0.600
4            1  0.410   0.300  settled   2.0      0.110
5            1  0.300   0.245  settled   1.0      0.055
6            1  0.245   0.100  settled   0.0      0.145
7            1  0.100  -0.500      bed   NaN      0.600


### Testing very thin bars

In [7]:
THIN = 0.100001

# df = pd.DataFrame(
#             {   
#                 "cell_number": [0,0,0,0,1,1,1,1],
#                 "top": [0.41, 0.3, 0.245, 0.1, 0.41, 0.3, THIN, 0.1],
#                 "bottom": [0.3, 0.245, 0.1, -0.5, 0.3, THIN, 0.1, -0.5],
#                 "type": ["settled", "settled", "settled", "bed", "settled", "settled", "settled", "bed"],
#                 "name": ["2", "1", "0", "existing", "2", "1", "0", "existing"],
#             }
#         )

df = pd.read_pickle('df.pickle') #a problematic dataframe

color_by = 'name'
color_by = 'proximity'

cm = Colormap('viridis')
df[color_by] = pd.to_numeric(df[color_by], errors="coerce")

n_colors = len(df[color_by].unique()) - 1
n_cells = len(df["cell_number"].unique())
colours = cm(np.linspace(0,1,n_colors))

fig = go.Figure()
for n, df1 in df.groupby(color_by, dropna=False):
    print(df1)
    if pd.isna(n):
        colour = ['#2e2e2e'] * n_cells
        name = 'seabed'
    else:
        colour = colours[int(n)]
        colour = [f"rgb{colour[0], colour[1], colour[2]}"] * n_cells
        name = f"Settled {int(n)}"
    
    fig.add_trace(
        go.Bar(
            x=df1["cell_number"],
            y=df1["thickness"],
            base=df1["bottom"],
            marker_color=colour,
            marker_line_width=0,
            width = 1,
            name=name,
            hovertemplate="Top: %{y:.5f}m"
            + "<br>Bottom: %{base:.5f}m"
            + "<br>Thickness: %{customdata[0]:,.5f}m"
            + f"<br>Name: {name}"
            + "<br><b>Cell</b>: %{x}<br>",
            customdata= df1[["thickness"]]
        )
    )

fig.update_layout(barmode="stack")#, bargap=0.0)
fig.show()

         top  bottom     type name  origin_cell  cell_number  thickness  \
44 -0.070833    -0.1  settled    0         22.0           22   0.029167   

    proximity  
44        0.0  
         top  bottom     type name  origin_cell  cell_number  thickness  \
42  0.013125     0.0  settled    0         22.0           21   0.013125   
46  0.013125     0.0  settled    0         22.0           23   0.013125   

    proximity  
42        1.0  
46        1.0  
         top  bottom     type name  origin_cell  cell_number  thickness  \
40  0.011813     0.0  settled    0         22.0           20   0.011813   
48  0.011813     0.0  settled    0         22.0           24   0.011813   

    proximity  
40        2.0  
48        2.0  
         top  bottom     type name  origin_cell  cell_number  thickness  \
38  0.010631     0.0  settled    0         22.0           19   0.010631   
50  0.010631     0.0  settled    0         22.0           25   0.010631   

    proximity  
38        3.0  
50        3